In [1]:
import xml.etree.ElementTree as ET

In [2]:
PC_count = False
inchi = False
inchikey = False  
hydrogen_bond_acceptor = False
hydrogen_bond_donor = False
rotatable_bond = False
iupac = False
logp = False
mass = False
molecular_formula = False
molecular_weight = False
smiles = False
topological = False
monoisotopic_weight = False
complexity = False


for event, elem in ET.iterparse("Compound_036500001_037000000.xml", events=("start","end")):
    prefix, has_namespace, postfix = elem.tag.partition('}')
    if has_namespace:
        elem.tag = postfix  # strip all namespaces
    if((elem.tag == "PC-CompoundType_id_cid") & (event == 'start')):     
        current_compound = {}
        compound_data = {}
        current_compound["_id"] = elem.text
        compound_data["cid"] = elem.text
        compound_data["iupac"] = {}
        compound_data["smiles"] = {}
    elif((elem.tag == "PC-Compound") & (event == 'end')):
        current_compound["pubchem"] = compound_data
        print(current_compound)
        elem.clear()
    elif((elem.tag == "PC-Compound_charge") & (event == 'start')):
        compound_data["formal_charge"] = elem.text
    elif((elem.tag == "PC-Count") & (event == 'start')):
        PC_count = True
    elif((elem.tag == "PC-Count") & (event == 'end')):
        PC_count = False
    elif(PC_count):
        if((elem.tag == "PC-Count_heavy-atom") & (event == 'start')):
            compound_data["heavy_atom_count"] = elem.text
        elif((elem.tag == "PC-Count_atom-chiral-def") & (event == 'start')):
            compound_data["defined_chiral_atom_count"] = elem.text
        elif((elem.tag == "PC-Count_bond-chiral-def") & (event == 'start')):
            compound_data["defined_chiral_bond_count"] = elem.text
        elif((elem.tag == "PC-Count_atom-chiral-undef") & (event == 'start')):
            compound_data["undefined_chiral_atom_count"] = elem.text
        elif((elem.tag == "PC-Count_bond-chiral-undef") & (event == 'start')):
            compound_data["undefined_chiral_bond_count"] = elem.text
        elif((elem.tag == "PC-Count_isotope-atom") & (event == 'start')):
            compound_data["isotope_atom_count"] = elem.text
        elif((elem.tag == "PC-Count_covalent-unit") & (event == 'start')):
            compound_data["covalent_unit_count"] = elem.text
        elif((elem.tag == "PC-Count_tautomers") & (event == 'start')):
            compound_data["tautomers_count"] = elem.text
    elif((elem.tag == "PC-Count") & (event == 'start')):
        PC_count = True
    elif((elem.tag == "PC-Count") & (event == 'end')):
        PC_count = False
        
    elif((elem.tag == "PC-Urn_label") & (event == 'start')):
        if(elem.text == 'InChI'):
            inchi = True
        elif(elem.text == 'InChIKey'):
            inchikey = True
        elif(elem.text == 'IUPAC Name'):
            iupac = True
        elif(elem.text == "Log P"):
            logp = True
        elif(elem.text == "Mass"):
            mass = True
        elif(elem.text == "Molecular Formula"):
            molecular_formula = True
        elif(elem.text == "Molecular Weight"):
            molecular_weight = True
        elif(elem.text == "SMILES"):
            smiles = True
        elif(elem.text == "Topological"):
            topological = True
        elif(elem.text == "Weight"):
            monoisotopic_weight = True
        elif(elem.text == "Compound Complexity"):
            complexity = True

    elif((elem.tag == "PC-Urn_name") & (event == 'start')):
        if(elem.text == 'Hydrogen Bond Acceptor'):
            hydrogen_bond_acceptor = True
        elif(elem.text == 'Hydrogen Bond Donor'):
            hydrogen_bond_donor = True
        elif(elem.text == 'Rotatable Bond'):
            rotatable_bond = True
        elif(iupac):
            iupac_key = elem.text  
        elif(smiles):
            smiles_key = elem.text  

    elif((elem.tag == "PC-InfoData_value_sval") & (event == 'start')):
        if(inchi):
            compound_data["inchi"] = elem.text
            inchi = False  
        elif(inchikey):
            compound_data["inchikey"] = elem.text
            inchikey = False  
        elif(iupac):
            compound_data['iupac'][iupac_key] = elem.text
            iupac = False
        elif(molecular_formula):
            compound_data["molecular_formula"] = elem.text
            molecular_formula = False
        elif(smiles):
            compound_data['smiles'][smiles_key] = elem.text
            smiles = False
    
    elif((elem.tag == "PC-InfoData_value_ival") & (event == 'start')):
        if(hydrogen_bond_acceptor):
            compound_data["hydrogen_bond_acceptor_count"] = elem.text
            hydrogen_bond_acceptor = False
        elif(hydrogen_bond_donor):
            compound_data["hydrogen_bond_donor_count"] = elem.text
            hydrogen_bond_donor = False
        elif(rotatable_bond):
            compound_data["rotatable_bond_count"] = elem.text
            rotatable_bond = False
        
    elif((elem.tag == "PC-InfoData_value_fval") & (event == 'start')):
        if(logp):
            compound_data["xlogp"] = elem.text
            logp = False
        elif(mass):
            compound_data["exact_mass"] = elem.text
            mass = False
        elif(molecular_weight): 
            compound_data["molecular_weight"] = elem.text
            molecular_weight = False
        elif(topological):
            compound_data["topological_polar_surface_area"] = elem.text
            topological = False
        elif(monoisotopic_weight):
            compound_data["monoisotopic_weight"] = elem.text
            monoisotopic_weight = False
        elif(complexity):
            compound_data["complexity"] = elem.text
            complexity = False
            
            
#     elem.clear()


{'_id': '36500004', 'pubchem': {'cid': '36500004', 'iupac': {'Allowed': '1-butyl-N-[4-(5-methyl-2-thienyl)thiazol-2-yl]-6-oxo-pyridazine-3-carboxamide', 'CAS-like Style': '1-butyl-N-[4-(5-methyl-2-thiophenyl)-2-thiazolyl]-6-oxo-3-pyridazinecarboxamide', 'Markup': '1-butyl-<I>N</I>-[4-(5-methylthiophen-2-yl)-1,3-thiazol-2-yl]-6-oxopyridazine-3-carboxamide', 'Preferred': '1-butyl-N-[4-(5-methylthiophen-2-yl)-1,3-thiazol-2-yl]-6-oxopyridazine-3-carboxamide', 'Systematic': '1-butyl-N-[4-(5-methylthiophen-2-yl)-1,3-thiazol-2-yl]-6-oxidanylidene-pyridazine-3-carboxamide', 'Traditional': '1-butyl-6-keto-N-[4-(5-methyl-2-thienyl)thiazol-2-yl]pyridazine-3-carboxamide'}, 'smiles': {'Canonical': 'CCCCN1C(=O)C=CC(=N1)C(=O)NC2=NC(=CS2)C3=CC=C(S3)C', 'Isomeric': 'CCCCN1C(=O)C=CC(=N1)C(=O)NC2=NC(=CS2)C3=CC=C(S3)C'}, 'formal_charge': '0', 'complexity': '581', 'hydrogen_bond_acceptor_count': '6', 'hydrogen_bond_donor_count': '1', 'rotatable_bond_count': '6', 'inchi': 'InChI=1S/C17H18N4O2S2/c1-3-4-9-21-

{'_id': '36501324', 'pubchem': {'cid': '36501324', 'iupac': {'Allowed': 'N-[4-[4-(acetamidomethyl)phenyl]thiazol-2-yl]-5-[(4-methoxyphenoxy)methyl]-2-methyl-furan-3-carboxamide', 'CAS-like Style': 'N-[4-[4-(acetamidomethyl)phenyl]-2-thiazolyl]-5-[(4-methoxyphenoxy)methyl]-2-methyl-3-furancarboxamide', 'Markup': '<I>N</I>-[4-[4-(acetamidomethyl)phenyl]-1,3-thiazol-2-yl]-5-[(4-methoxyphenoxy)methyl]-2-methylfuran-3-carboxamide', 'Preferred': 'N-[4-[4-(acetamidomethyl)phenyl]-1,3-thiazol-2-yl]-5-[(4-methoxyphenoxy)methyl]-2-methylfuran-3-carboxamide', 'Systematic': 'N-[4-[4-(acetamidomethyl)phenyl]-1,3-thiazol-2-yl]-5-[(4-methoxyphenoxy)methyl]-2-methyl-furan-3-carboxamide', 'Traditional': 'N-[4-[4-(acetamidomethyl)phenyl]thiazol-2-yl]-5-[(4-methoxyphenoxy)methyl]-2-methyl-3-furamide'}, 'smiles': {'Canonical': 'CC1=C(C=C(O1)COC2=CC=C(C=C2)OC)C(=O)NC3=NC(=CS3)C4=CC=C(C=C4)CNC(=O)C', 'Isomeric': 'CC1=C(C=C(O1)COC2=CC=C(C=C2)OC)C(=O)NC3=NC(=CS3)C4=CC=C(C=C4)CNC(=O)C'}, 'formal_charge': '0', 

{'_id': '36503883', 'pubchem': {'cid': '36503883', 'iupac': {'Allowed': 'methyl 5-[methyl-[[4-(trifluoromethyl)phenyl]methyl]amino]-2-nitro-benzoate', 'CAS-like Style': '5-[methyl-[[4-(trifluoromethyl)phenyl]methyl]amino]-2-nitrobenzoic acid methyl ester', 'Markup': 'methyl 5-[methyl-[[4-(trifluoromethyl)phenyl]methyl]amino]-2-nitrobenzoate', 'Preferred': 'methyl 5-[methyl-[[4-(trifluoromethyl)phenyl]methyl]amino]-2-nitrobenzoate', 'Systematic': 'methyl 5-[methyl-[[4-(trifluoromethyl)phenyl]methyl]amino]-2-nitro-benzoate', 'Traditional': '5-[methyl-[4-(trifluoromethyl)benzyl]amino]-2-nitro-benzoic acid methyl ester'}, 'smiles': {'Canonical': 'CN(CC1=CC=C(C=C1)C(F)(F)F)C2=CC(=C(C=C2)[N+](=O)[O-])C(=O)OC', 'Isomeric': 'CN(CC1=CC=C(C=C1)C(F)(F)F)C2=CC(=C(C=C2)[N+](=O)[O-])C(=O)OC'}, 'formal_charge': '0', 'complexity': '501', 'hydrogen_bond_acceptor_count': '8', 'hydrogen_bond_donor_count': '0', 'rotatable_bond_count': '5', 'inchi': 'InChI=1S/C17H15F3N2O4/c1-21(10-11-3-5-12(6-4-11)17(18,19

{'_id': '36505276', 'pubchem': {'cid': '36505276', 'iupac': {'Allowed': '[4-(3-bromobenzoyl)piperazin-1-yl]-(4-fluorophenyl)methanone', 'CAS-like Style': '[4-[(3-bromophenyl)-oxomethyl]-1-piperazinyl]-(4-fluorophenyl)methanone', 'Markup': '[4-(3-bromobenzoyl)piperazin-1-yl]-(4-fluorophenyl)methanone', 'Preferred': '[4-(3-bromobenzoyl)piperazin-1-yl]-(4-fluorophenyl)methanone', 'Systematic': '[4-(3-bromophenyl)carbonylpiperazin-1-yl]-(4-fluorophenyl)methanone', 'Traditional': '[4-(3-bromobenzoyl)piperazino]-(4-fluorophenyl)methanone'}, 'smiles': {'Canonical': 'C1CN(CCN1C(=O)C2=CC=C(C=C2)F)C(=O)C3=CC(=CC=C3)Br', 'Isomeric': 'C1CN(CCN1C(=O)C2=CC=C(C=C2)F)C(=O)C3=CC(=CC=C3)Br'}, 'formal_charge': '0', 'complexity': '460', 'hydrogen_bond_acceptor_count': '3', 'hydrogen_bond_donor_count': '0', 'rotatable_bond_count': '2', 'inchi': 'InChI=1S/C18H16BrFN2O2/c19-15-3-1-2-14(12-15)18(24)22-10-8-21(9-11-22)17(23)13-4-6-16(20)7-5-13/h1-7,12H,8-11H2', 'inchikey': 'OYZMEQAHRLCEEF-UHFFFAOYSA-N', 'xlogp

{'_id': '36505667', 'pubchem': {'cid': '36505667', 'iupac': {'Allowed': '3-cyano-N-[(1R)-1-[5-methyl-1-[3-(trifluoromethyl)phenyl]pyrazol-4-yl]ethyl]benzamide', 'CAS-like Style': '3-cyano-N-[(1R)-1-[5-methyl-1-[3-(trifluoromethyl)phenyl]-4-pyrazolyl]ethyl]benzamide', 'Markup': '3-cyano-<I>N</I>-[(1<I>R</I>)-1-[5-methyl-1-[3-(trifluoromethyl)phenyl]pyrazol-4-yl]ethyl]benzamide', 'Preferred': '3-cyano-N-[(1R)-1-[5-methyl-1-[3-(trifluoromethyl)phenyl]pyrazol-4-yl]ethyl]benzamide', 'Systematic': '3-cyano-N-[(1R)-1-[5-methyl-1-[3-(trifluoromethyl)phenyl]pyrazol-4-yl]ethyl]benzamide', 'Traditional': None}, 'smiles': {'Canonical': 'CC1=C(C=NN1C2=CC=CC(=C2)C(F)(F)F)C(C)NC(=O)C3=CC=CC(=C3)C#N', 'Isomeric': 'CC1=C(C=NN1C2=CC=CC(=C2)C(F)(F)F)[C@@H](C)NC(=O)C3=CC=CC(=C3)C#N'}, 'formal_charge': '0', 'complexity': '631', 'hydrogen_bond_acceptor_count': '6', 'hydrogen_bond_donor_count': '1', 'rotatable_bond_count': '4', 'inchi': 'InChI=1S/C21H17F3N4O/c1-13(27-20(29)16-6-3-5-15(9-16)11-25)19-12-26-28(

{'_id': '36506088', 'pubchem': {'cid': '36506088', 'iupac': {'Allowed': '1-[2-(4-chlorophenoxy)ethyl]-3-[(R)-(4-chlorophenyl)-(1-methylimidazol-2-yl)methyl]urea', 'CAS-like Style': '1-[2-(4-chlorophenoxy)ethyl]-3-[(R)-(4-chlorophenyl)-(1-methyl-2-imidazolyl)methyl]urea', 'Markup': '1-[2-(4-chlorophenoxy)ethyl]-3-[(<I>R</I>)-(4-chlorophenyl)-(1-methylimidazol-2-yl)methyl]urea', 'Preferred': '1-[2-(4-chlorophenoxy)ethyl]-3-[(R)-(4-chlorophenyl)-(1-methylimidazol-2-yl)methyl]urea', 'Systematic': '1-[2-(4-chloranylphenoxy)ethyl]-3-[(R)-(4-chlorophenyl)-(1-methylimidazol-2-yl)methyl]urea', 'Traditional': '1-[2-(4-chlorophenoxy)ethyl]-3-[(R)-(4-chlorophenyl)-(1-methylimidazol-2-yl)methyl]urea'}, 'smiles': {'Canonical': 'CN1C=CN=C1C(C2=CC=C(C=C2)Cl)NC(=O)NCCOC3=CC=C(C=C3)Cl', 'Isomeric': 'CN1C=CN=C1[C@@H](C2=CC=C(C=C2)Cl)NC(=O)NCCOC3=CC=C(C=C3)Cl'}, 'formal_charge': '0', 'complexity': '483', 'hydrogen_bond_acceptor_count': '3', 'hydrogen_bond_donor_count': '2', 'rotatable_bond_count': '7', 'i

{'_id': '36506967', 'pubchem': {'cid': '36506967', 'iupac': {'Allowed': '1-(2-furylmethyl)-3-[3-(3-methylphenoxy)propyl]urea', 'CAS-like Style': '1-(2-furanylmethyl)-3-[3-(3-methylphenoxy)propyl]urea', 'Markup': '1-(furan-2-ylmethyl)-3-[3-(3-methylphenoxy)propyl]urea', 'Preferred': '1-(furan-2-ylmethyl)-3-[3-(3-methylphenoxy)propyl]urea', 'Systematic': '1-(furan-2-ylmethyl)-3-[3-(3-methylphenoxy)propyl]urea', 'Traditional': '1-(2-furfuryl)-3-[3-(3-methylphenoxy)propyl]urea'}, 'smiles': {'Canonical': 'CC1=CC(=CC=C1)OCCCNC(=O)NCC2=CC=CO2', 'Isomeric': 'CC1=CC(=CC=C1)OCCCNC(=O)NCC2=CC=CO2'}, 'formal_charge': '0', 'complexity': '312', 'hydrogen_bond_acceptor_count': '3', 'hydrogen_bond_donor_count': '2', 'rotatable_bond_count': '7', 'inchi': 'InChI=1S/C16H20N2O3/c1-13-5-2-6-14(11-13)20-10-4-8-17-16(19)18-12-15-7-3-9-21-15/h2-3,5-7,9,11H,4,8,10,12H2,1H3,(H2,17,18,19)', 'inchikey': 'NIPHKSWKFAOUIF-UHFFFAOYSA-N', 'xlogp': '2.4', 'exact_mass': '288.14739251', 'molecular_formula': 'C16H20N2O3',

{'_id': '36507495', 'pubchem': {'cid': '36507495', 'iupac': {'Allowed': '1-[(S)-(2-fluorophenyl)-(1-methylimidazol-2-yl)methyl]-3-[2-(4-methoxyphenyl)ethyl]urea', 'CAS-like Style': '1-[(S)-(2-fluorophenyl)-(1-methyl-2-imidazolyl)methyl]-3-[2-(4-methoxyphenyl)ethyl]urea', 'Markup': '1-[(<I>S</I>)-(2-fluorophenyl)-(1-methylimidazol-2-yl)methyl]-3-[2-(4-methoxyphenyl)ethyl]urea', 'Preferred': '1-[(S)-(2-fluorophenyl)-(1-methylimidazol-2-yl)methyl]-3-[2-(4-methoxyphenyl)ethyl]urea', 'Systematic': '1-[(S)-(2-fluorophenyl)-(1-methylimidazol-2-yl)methyl]-3-[2-(4-methoxyphenyl)ethyl]urea', 'Traditional': '1-[(S)-(2-fluorophenyl)-(1-methylimidazol-2-yl)methyl]-3-[2-(4-methoxyphenyl)ethyl]urea'}, 'smiles': {'Canonical': 'CN1C=CN=C1C(C2=CC=CC=C2F)NC(=O)NCCC3=CC=C(C=C3)OC', 'Isomeric': 'CN1C=CN=C1[C@H](C2=CC=CC=C2F)NC(=O)NCCC3=CC=C(C=C3)OC'}, 'formal_charge': '0', 'complexity': '487', 'hydrogen_bond_acceptor_count': '4', 'hydrogen_bond_donor_count': '2', 'rotatable_bond_count': '7', 'inchi': 'InCh

{'_id': '36508015', 'pubchem': {'cid': '36508015', 'iupac': {'Allowed': '1-[(1S)-1-methyl-3-phenyl-propyl]-3-(1-methylsulfonyl-4-piperidyl)urea', 'CAS-like Style': '1-(1-methylsulfonyl-4-piperidinyl)-3-[(2S)-4-phenylbutan-2-yl]urea', 'Markup': '1-(1-methylsulfonylpiperidin-4-yl)-3-[(2<I>S</I>)-4-phenylbutan-2-yl]urea', 'Preferred': '1-(1-methylsulfonylpiperidin-4-yl)-3-[(2S)-4-phenylbutan-2-yl]urea', 'Systematic': '1-(1-methylsulfonylpiperidin-4-yl)-3-[(2S)-4-phenylbutan-2-yl]urea', 'Traditional': '1-(1-mesyl-4-piperidyl)-3-[(1S)-1-methyl-3-phenyl-propyl]urea'}, 'smiles': {'Canonical': 'CC(CCC1=CC=CC=C1)NC(=O)NC2CCN(CC2)S(=O)(=O)C', 'Isomeric': 'C[C@@H](CCC1=CC=CC=C1)NC(=O)NC2CCN(CC2)S(=O)(=O)C'}, 'formal_charge': '0', 'complexity': '490', 'hydrogen_bond_acceptor_count': '4', 'hydrogen_bond_donor_count': '2', 'rotatable_bond_count': '6', 'inchi': 'InChI=1S/C17H27N3O3S/c1-14(8-9-15-6-4-3-5-7-15)18-17(21)19-16-10-12-20(13-11-16)24(2,22)23/h3-7,14,16H,8-13H2,1-2H3,(H2,18,19,21)/t14-/m0/s1

KeyboardInterrupt: 